In [1]:
import sys
sys.path.insert(0, '../../src')
import feedparser
import pandas as pd

import urllib

from postagem.Util import extract_domain, download_and_move_image, get_noticia_comercio
from lexical_analyzer_package import midia_lexical
from midia_postagem import midia_post
from Model.News import News
from Database import midia_table

from newsplease import NewsPlease
from bs4 import BeautifulSoup
import requests

import datetime

In [2]:
link = 'https://www.revistaforum.com.br/politica/'

In [3]:
req = requests.get(link)

In [4]:
noticias = BeautifulSoup(req.text, "html.parser").find_all('h4', class_='media-heading')

In [ ]:
for noticia in noticias:
    print(noticia.find_all('a', href=True)[0]['href'])
    article = NewsPlease.from_url(noticia.find_all('a', href=True)[0]['href'])
    row = {'titulos': [], 'links': [], 'noticia': [], 'image': [], 'abstract': [], 'date': []}
    if (article is not None):
        row['titulos'].append(article.title)
        row['noticia'].append(article.text)
        row['links'].append(article.url)
        row['abstract'].append(article.text)
        row['date'].append(article.date_publish)
        path_image = article.image_url
        print(path_image)
        if path_image == '' or path_image == None:
            row['image'].append(0)
        else:
            row['image'].append(download_and_move_image(article.image_url))
        news = News(row['abstract'], row['noticia'], row['date'], row['links'], row['titulos'], row['image'])
        try:
            print(row['titulos'])
            news_in_db = midia_table.check_news(news)
            print('news_in_db: ' + str(news_in_db))
            if (not news_in_db):
                row = pd.DataFrame(row)
                df, categories = midia_lexical.lexical_corpus_and_title(row)
                # DB categories
                if (categories != [set()]):
                    news.set_categories(categories)
                    midia_table.save_news(news)
                    midia_post.post_news(df)
        except:
            print('Empty News')

https://www.revistaforum.com.br/policiais-e-militares-eleitos-para-o-legislativo-aumentam-de-18-para-73/
https://www.revistaforum.com.br/wp-content/uploads/2018/10/urnas.jpg
100% [..............................................................................] 38384 / 38384urnas.jpg
C:\Users\admin\rss3\src\midia_postagem\images\urnas.jpg
['Policiais e militares eleitos para o Legislativo aumentam de 18 para 73']
news_in_db: False
 -- no categories -- 
https://www.revistaforum.com.br/nunca-comemorei-tanto-um-erro-do-meu-pai-diz-filho-de-noblat/
https://www.revistaforum.com.br/wp-content/uploads/2018/10/guganoblat.png
100% [..............................................................................] 24538 / 24538guganoblat.png
C:\Users\admin\rss3\src\midia_postagem\images\guganoblat.png
['"Nunca comemorei tanto um erro do meu pai", diz filho de Noblat']
news_in_db: False
 -- no categories -- 
https://www.revistaforum.com.br/trabalhadores-dos-eua-enviam-mais-de-700-cartoes-postais-para-

In [ ]:
noticias[0]